# Synapse Analytics Benchmarks - Radius

### Background

Radius is a network telemetry format used in the Telecommunications industry. In 2016 endjin developed a solution on Azure for TalkTalk to enable them to rapidly ingest and process Radius packets and generate insights into the health of the network and the impact to customers.

The solution was developed using Azure Data Lake Analytics which is no longer being actively developed by Microsoft so an alternative needs to be found.

Azure Synapse SQL Serverless is a new offering. Like Azure Data Lake Analytics, SQL Serverless is SQL-based and provides query and data processing on a consumption basis so it feels like it could be a good option.

### Experiment

It is possible to replace Azure Data Lake Analytics with Azure Synapse SQL Serverless for the Talk Talk network health analytics solution.

In order to be considered a replacement it's important to understand the performance charateristics of the service with regard to various factors such as representative data volumes, file sizes and formats. Performance should be comparable with ADLA.

Since SQL Serverless does not support querying the native Radius format data will need to be pre-processed into a format that SQL Serverless supports. Formats under consideration are:

- Parquet
- CSV
- CSV 2.0 (MisonCSV)

## Define test parameters

In [1]:

var server = "endjin-ondemand.sql.azuresynapse.net";
var username = "BenchmarkTool";
var _password = password("Enter the password for the BenchmarkTool user");

The below script needs to be able to find the current output cell; this is an easy method to get it.

Enter the password for the BenchmarkTool user ·······


In [2]:
#r "nuget:System.Data.SqlClient"
#r "nuget:Microsoft.Data.Analysis,0.2.0"
#r "nuget:Newtonsoft.Json,12.0.3"
    
using System.Data;
using System.Data.SqlClient;
using System.Diagnostics;
using Microsoft.Data.Analysis;
using System.IO;
using Newtonsoft.Json;


Installed package Microsoft.Data.Analysis version 0.2.0

Installed package Newtonsoft.Json version 12.0.3

Installed package System.Data.SqlClient version 4.8.1

In [3]:
var builder = new SqlConnectionStringBuilder($"Server=(local);Persist Security Info=False;Initial Catalog=Radius");
builder["Server"] = server;
builder.Password = _password.GetClearTextPassword();
builder["UID"] = username;
builder["Connect Timeout"] = 1000;
builder["Trusted_Connection"] = false;

In [4]:
using Microsoft.AspNetCore.Html;
Formatter<DataFrame>.Register((df, writer) =>
{
    var headers = new List<IHtmlContent>();
    headers.Add(th(i("index")));
    headers.AddRange(df.Columns.Select(c => (IHtmlContent) th(c.Name)));
    var rows = new List<List<IHtmlContent>>();
    var take = 20;
    for (var i = 0; i < Math.Min(take, df.Rows.Count); i++)
    {
        var cells = new List<IHtmlContent>();
        cells.Add(td(i));
        foreach (var obj in df.Rows[i])
        {
            cells.Add(td(obj));
        }
        rows.Add(cells);
    }
    
    var t = table(
        thead(
            headers),
        tbody(
            rows.Select(
                r => tr(r))));
    
    writer.Write(t);
}, "text/html");

### Classes to run benchmarks

In [5]:
public class RadiusTest
{
    public string Name { get; set; }
    public string Query { get; set; }
}

public class RadiusTestResult
{
    public string Name { get; set; }
    public int RunNumber { get; set; }
    public double DurationInSeconds { get; set; }
}

public class TestRunner
{
    private string connectionString;

    public TestRunner(string connectionString)
    {
        this.connectionString = connectionString;
    }

    public RadiusTestResult Run(RadiusTest test, int runNumber)
    {
        using (var connection = new SqlConnection(this.connectionString))
        {
            connection.Open();
            var stopwatch = new Stopwatch();

            stopwatch.Reset();
            stopwatch.Start();
            Run(connection, test.Query);
            stopwatch.Stop();
            return new RadiusTestResult { Name = test.Name, RunNumber = runNumber, DurationInSeconds = stopwatch.ElapsedMilliseconds / 1000d };
        }
    }

    public List<RadiusTestResult> RunSeries(RadiusTest test, int numberOfIterations)
    {        
        var results = new List<RadiusTestResult>();
        for (int i = 0; i < numberOfIterations; i++)
        {
            results.Add(Run(test, i + 1));
        }

        return results;
    }   

    public List<RadiusTestResult> RunParallel(RadiusTest test, int numberOfIterations)
    {
        var results = Task.WhenAll(Enumerable
            .Range(0, numberOfIterations)
            .Select(i => Task.Run(() => Run(test, i + 1)))).Result;

        return results.ToList();
    }

    private void Run(SqlConnection connection, string query)
    {
        SqlCommand command = new SqlCommand(query, connection);
        command.CommandTimeout = 18000;

        using (SqlDataReader reader = command.ExecuteReader())
        {
            reader.Read();
        }
    }
}

In [6]:
using XPlot.Plotly;
using System.Linq;

In [7]:
var testRunner = new TestRunner(builder.ConnectionString);

void Plot(IEnumerable<RadiusTestResult> results, string name)
{
    var chart = Chart.Plot(
        new [] {
            new Graph.Bar()
            {
                x = results.Select(o => "Run " + o.RunNumber),
                y = results.Select(o => o.DurationInSeconds),
            }
        }
    );
    
    var average = results.Average(o => o.DurationInSeconds);
    
    var layout = new Layout.Layout {
        shapes = new [] { new Graph.Shape { x0 = 0, y0 = average, x1 = results.Count() - 1, y1 = average, opacity = 0.5 } }
    };

    chart.WithLayout(layout);
    chart.WithTitle("Benchmark: " + name);
    chart.WithYTitle("Duration in seconds");
    display(chart);
}


enum TestType
{
    Series, 
    Parallel
}


IList<RadiusTestResult> RunBenchmark(string name, string query, TestType testType, bool force = false)
{
    IList<RadiusTestResult> results = null;
    
    if (!force)
    {
        if (File.Exists(name + ".json"))
        {
            results = JsonConvert.DeserializeObject<IList<RadiusTestResult>>(File.ReadAllText(name + ".json"));
        }
    }
    
    if (results == null)
    {
        var test = new RadiusTest { Name = name, Query = query };
        
        display("Running " + name + "...");


        switch (testType)
        {
            case TestType.Series: 
                results = testRunner.RunSeries(test, 10);
                break;
            case TestType.Parallel:
                results = testRunner.RunParallel(test, 10);
                break;
            default:
                throw new ArgumentException("Invalid testType");
        }

        File.WriteAllText(name + ".json", JsonConvert.SerializeObject(results));
    }
    
    display(results);
    Plot(results, name);
    return results;
}

## Run benchmarks

### Method

A pre-existing Azure Data Lake Analytics query was ported over to SQL Serverless.

To replicate the real-world scenario the query was decomposed into a set of views. The views read raw data from the data lake and progressively perform various transformations including data type conversions and aggregations.

Each set of views were replicated to work with each parser under test. The following parsers were used:

- Parquet
- CSV
- CSV 2.0

Raw data was pre-parsed into both Parquet and CSV formats and stored as a single file (~ 2GB) as well as separate hourly partitions.

![Files partitioned by hour](files-partitioned-by-hour.png)

The query was executed 10 times against each combination of parser and file source, once in series and once with all 10 queries running concurrently.

This equates to a total of 12 invidual test runs.

| Parser | Source | Execution |
|-|-|-|
| Parquet | Single File | Series |
| Parquet | Single File | Parallel |
| Parquet | Muliple Files | Series |
| Parquet | Multiple Files | Parallel |
| CSV | Single File | Series |
| CSV | Single File | Parallel |
| CSV | Muliple Files | Series |
| CSV | Multiple Files | Parallel |
| CSV 2.0 | Single File | Series |
| CSV 2.0 | Single File | Parallel |
| CSV 2.0 | Muliple Files | Series |
| CSV 2.0 | Multiple Files | Parallel |

### Running the benchmarks

Each test is defined in it's own cell.

For consistent results it's recommended to execute the entire notebook rather than running individual cells. This ensures there is negligable time between tests and avoids unintended cold starts.

All benchmark results are automatically persisted. This not only allows a point in time snapshot to be recorded and maintain between notebook kernel restrats but also can be useful when working with the results of a previous run.

To force the re-execution of the benchmaks then set `force` variable below to `true` and then re-run the notebook. When `force` is `false` the notebook will will first check to see if persisted results already exist and will return these, otherwise then the benchmark will be re-executed.

In [8]:
var force = false;

### Parquet, Single File, Series

In [9]:
var parquetSingleFileSeriesResults = RunBenchmark("Parquet, Single File, Series", "SELECT * FROM NetworkHealth_Parquet_SingleFile", TestType.Series, force);

index,Name,RunNumber,DurationInSeconds
0,"Parquet, Single File, Series",1,26.917
1,"Parquet, Single File, Series",2,23.188
2,"Parquet, Single File, Series",3,24.902
3,"Parquet, Single File, Series",4,25.287
4,"Parquet, Single File, Series",5,28.016
5,"Parquet, Single File, Series",6,29.495
6,"Parquet, Single File, Series",7,29.909
7,"Parquet, Single File, Series",8,29.881
8,"Parquet, Single File, Series",9,31.289
9,"Parquet, Single File, Series",10,30.315


### Observations

The average duration was ~ 27 seconds per run and the performance was consistent. There was no noticable cold start up times.

### Parquet, Single File, Parallel

In [10]:
var parquetSingleFileParallelResults = RunBenchmark("Parquet, Single File, Parallel", "SELECT * FROM NetworkHealth_Parquet_SingleFile", TestType.Parallel, force);

index,Name,RunNumber,DurationInSeconds
0,"Parquet, Single File, Parallel",1,32.062
1,"Parquet, Single File, Parallel",2,40.851
2,"Parquet, Single File, Parallel",3,57.786
3,"Parquet, Single File, Parallel",4,76.208
4,"Parquet, Single File, Parallel",5,87.572
5,"Parquet, Single File, Parallel",6,104.077
6,"Parquet, Single File, Parallel",7,138.122
7,"Parquet, Single File, Parallel",8,120.539
8,"Parquet, Single File, Parallel",9,151.543
9,"Parquet, Single File, Parallel",10,167.31


It appears that the time it takes is increasing linearly based on the number of runs. Re-order to be sure.

In [11]:
var r = parquetSingleFileParallelResults.OrderBy(o => o.DurationInSeconds).ToList();
Plot(r, "Parquet, Single File, Parallel - Ordered by duration");

### Observations

Parallel runs shows a steady increase in execution time - we can see this more clearly when ordering by duration since parallelizing will naturally randomise the execution order.
This indicates that there is resource contention when running a single file. When raising this with Microsoft they confirmed that internally that there are inefficiencies when running multiple queries over the same file at the same point in time. It is likely that Synapse is taking advantage of local caching for more efficient processing.

In the real-world this may or may not be a problem. One scenario where this could be an issue is if you are performing lots of processing in response to new files arriving. Multiple pipelines may trigger at the same time causing contention. What isn't clear is whether this behaviour is a result of performing *any* query against a single file or the *same* query against a single file. This requires further investigation.

**TODO**: Experiment to determine whether single file resource contention is governed by the file or the query.

**Feedback to Microsoft:** Contention when running queries concurrently against the same file may cause issues in production scenarios, especially for architectures that have responsive data processing in response to new files arriving. Is is possible to detect this condition and provide more uniform performance?
    

### Parquet, Mulitple Files, Series

In [12]:
var parquetMultipleFilesSeriesResults = RunBenchmark("Parquet, Multiple Files, Series", "SELECT * FROM NetworkHealth_Parquet_MultipleFiles", TestType.Series, force);

index,Name,RunNumber,DurationInSeconds
0,"Parquet, Multiple Files, Series",1,40.874
1,"Parquet, Multiple Files, Series",2,16.045
2,"Parquet, Multiple Files, Series",3,15.454
3,"Parquet, Multiple Files, Series",4,15.391
4,"Parquet, Multiple Files, Series",5,16.79
5,"Parquet, Multiple Files, Series",6,16.034
6,"Parquet, Multiple Files, Series",7,15.8
7,"Parquet, Multiple Files, Series",8,15.847
8,"Parquet, Multiple Files, Series",9,16.007
9,"Parquet, Multiple Files, Series",10,16.338


### Observations

The average duration was ~ 17 seconds, which was 10 seconds faster than the equivillent single file benchmark indicating that there is a significant performance advantage using smaller files.

The first run took considerably longer than the rest. The test was executed immediately after the first so this was not as a result of a cold start.

It's not particularly clear why the intial run took longer than the others.

It is known that SQL Serverless generates statistics for Parquet files as queries are run, so it's possible that this initial time can be attributed to this.

### Parquet, Mulitple Files, Parallel

In [13]:
var parquetMultipleFilesParallelResults = RunBenchmark("Parquet, Multiple Files, Parallel", "SELECT * FROM NetworkHealth_Parquet_MultipleFiles", TestType.Parallel, force);

index,Name,RunNumber,DurationInSeconds
0,"Parquet, Multiple Files, Parallel",1,23.23
1,"Parquet, Multiple Files, Parallel",2,22.899
2,"Parquet, Multiple Files, Parallel",3,23.263
3,"Parquet, Multiple Files, Parallel",4,19.919
4,"Parquet, Multiple Files, Parallel",5,25.76
5,"Parquet, Multiple Files, Parallel",6,24.662
6,"Parquet, Multiple Files, Parallel",7,46.824
7,"Parquet, Multiple Files, Parallel",8,34.809
8,"Parquet, Multiple Files, Parallel",9,28.789
9,"Parquet, Multiple Files, Parallel",10,40.313


The average duration was ~ 29 seconds. 

Order by duration:

In [14]:
var r = parquetMultipleFilesParallelResults.OrderBy(o => o.DurationInSeconds).ToList();
Plot(r, "Parquet, Multiple Files, Parallel - Ordered by duration");

### Observations

The performance profile for parallel execution again indicates resource contention, only this time it appears to be less severe than when quering a single file in parallel. We appear to get better utilisation for the first sets of queries (Runs 4, 2, 1, 3, 6 and 5) before there is a gradual tail off in performance. Since we know that multiple smaller files offer better performance it follows that resources are able to process more queries before the effect of contention starts to becomes apparent.

### CSV, Single File, Series

In [15]:
var csvSingleFileSeriesResults = RunBenchmark("CSV, Single File, Series", "SELECT * FROM NetworkHealth_CSV_SingleFile", TestType.Series, force);

index,Name,RunNumber,DurationInSeconds
0,"CSV, Single File, Series",1,101.949
1,"CSV, Single File, Series",2,102.966
2,"CSV, Single File, Series",3,102.17
3,"CSV, Single File, Series",4,101.792
4,"CSV, Single File, Series",5,101.293
5,"CSV, Single File, Series",6,101.46
6,"CSV, Single File, Series",7,101.577
7,"CSV, Single File, Series",8,100.892
8,"CSV, Single File, Series",9,102.088
9,"CSV, Single File, Series",10,101.918


### Observations

CSV offers very consistent peformance at ~ 100 seconds per query.

Query execution time significantly higher than for Parquet. 

During prevous test runs CSV would run faster than Parquet. This was counter intuative since SQL Serverless can use Parquet's statistics to generate more efficient queries.
This was a reported to Microsoft who **confirmed that this was a bug**; CSV queries were incorrectly allocated more resources than Parquet and causing queries to return faster.

Here we can see the average duration for CSV (~ 100 seconds) is now longer than for Parquet (~ 30s) which is more representative of what we would expected. We therefore conclude that this has now been fixed.

### CSV, Single File, Parallel

In [16]:
var csvSingleFileParallelResults = RunBenchmark("CSV, Single File, Parallel", "SELECT * FROM NetworkHealth_CSV_SingleFile", TestType.Parallel, force);

index,Name,RunNumber,DurationInSeconds
0,"CSV, Single File, Parallel",1,112.773
1,"CSV, Single File, Parallel",2,112.966
2,"CSV, Single File, Parallel",3,108.718
3,"CSV, Single File, Parallel",4,199.517
4,"CSV, Single File, Parallel",5,204.688
5,"CSV, Single File, Parallel",6,212.055
6,"CSV, Single File, Parallel",7,301.973
7,"CSV, Single File, Parallel",8,311.552
8,"CSV, Single File, Parallel",9,387.052
9,"CSV, Single File, Parallel",10,307.699


Order by duration

In [17]:
var r = csvSingleFileParallelResults.OrderBy(o => o.DurationInSeconds).ToList();
Plot(r, "CSV, Single File, Parallel - Ordered by duration");

### Observations

We see a distinct 'stepped' profile for CSV files. Again this is probably attributed to resource contention, but perhaps indicates jobs are queued and processed in strict batches. It's not clear why this is different to the Parquet profile.

### CSV, Multiple Files, Series

In [18]:
var csvMultipleFilesSeriesResults = RunBenchmark("CSV, Multiple Files, Series", "SELECT * FROM NetworkHealth_CSV_MultipleFiles", TestType.Series, force);

index,Name,RunNumber,DurationInSeconds
0,"CSV, Multiple Files, Series",1,24.173
1,"CSV, Multiple Files, Series",2,23.06
2,"CSV, Multiple Files, Series",3,21.842
3,"CSV, Multiple Files, Series",4,21.789
4,"CSV, Multiple Files, Series",5,22.878
5,"CSV, Multiple Files, Series",6,22.962
6,"CSV, Multiple Files, Series",7,23.627
7,"CSV, Multiple Files, Series",8,23.333
8,"CSV, Multiple Files, Series",9,23.025
9,"CSV, Multiple Files, Series",10,22.396


### Observations

Again we see consistent timings for CSV when run in series and better overall performance when running against multiple smaller files.

### CSV, Multiple Files, Parallel

In [19]:
var csvMultipleFilesParallelResults = RunBenchmark("CSV, Multiple Files, Parallel", "SELECT * FROM NetworkHealth_CSV_MultipleFiles", TestType.Parallel, force);

index,Name,RunNumber,DurationInSeconds
0,"CSV, Multiple Files, Parallel",1,34.803
1,"CSV, Multiple Files, Parallel",2,49.949
2,"CSV, Multiple Files, Parallel",3,35.545
3,"CSV, Multiple Files, Parallel",4,39.86
4,"CSV, Multiple Files, Parallel",5,38.06
5,"CSV, Multiple Files, Parallel",6,43.19
6,"CSV, Multiple Files, Parallel",7,54.644
7,"CSV, Multiple Files, Parallel",8,53.038
8,"CSV, Multiple Files, Parallel",9,63.416
9,"CSV, Multiple Files, Parallel",10,64.406


Order by duration

In [20]:
var r = csvMultipleFilesParallelResults.OrderBy(o => o.DurationInSeconds).ToList();
Plot(r, "CSV, Multiple Files, Parallel - Ordered by duration");

### Observations

We see a similar profile to parquet. Interestingly, we don't observe the noticable 'stepped' profile as we did when run over a single file. This perhaps suggests that Syanapse uses the underlying file size to determine how work is processed.

### CSV 2.0, Single File, Series

In [21]:
var csv20SingleFileSeriesResults = RunBenchmark("CSV 2.0, Single File, Series", "SELECT * FROM NetworkHealth_MisonCSV_SingleFile", TestType.Series, force);

index,Name,RunNumber,DurationInSeconds
0,"CSV 2.0, Single File, Series",1,22.72
1,"CSV 2.0, Single File, Series",2,23.72
2,"CSV 2.0, Single File, Series",3,24.809
3,"CSV 2.0, Single File, Series",4,22.102
4,"CSV 2.0, Single File, Series",5,21.817
5,"CSV 2.0, Single File, Series",6,22.29
6,"CSV 2.0, Single File, Series",7,22.895
7,"CSV 2.0, Single File, Series",8,22.801
8,"CSV 2.0, Single File, Series",9,22.854
9,"CSV 2.0, Single File, Series",10,22.667


### Observations

CSV 2.0 was faster at processing single file in series than Parquet by approximately 5 seconds. This was unexpected since it's understood that CSV 2.0 is not able to take advantage of statistics.

It's important to not that the query being executed reads all columns, so it's possible that other queries that read a subset of columns perform better with Parquet.

**TODO:** Run an experiment to validate whether Parquet out performs CSV 2.0 when using a subset of columns.

### CSV 2.0, Single File, Parallel

In [22]:
var csv20SingleFileParallelResults = RunBenchmark("CSV 2.0, Single File, Parallel", "SELECT * FROM NetworkHealth_MisonCSV_SingleFile", TestType.Parallel, force);

index,Name,RunNumber,DurationInSeconds
0,"CSV 2.0, Single File, Parallel",1,28.181
1,"CSV 2.0, Single File, Parallel",2,28.2
2,"CSV 2.0, Single File, Parallel",3,37.912
3,"CSV 2.0, Single File, Parallel",4,44.855
4,"CSV 2.0, Single File, Parallel",5,57.222
5,"CSV 2.0, Single File, Parallel",6,63.521
6,"CSV 2.0, Single File, Parallel",7,88.211
7,"CSV 2.0, Single File, Parallel",8,82.783
8,"CSV 2.0, Single File, Parallel",9,103.505
9,"CSV 2.0, Single File, Parallel",10,96.133


Order by duration

In [23]:
var r = csv20SingleFileParallelResults.OrderBy(o => o.DurationInSeconds).ToList();
Plot(r, "CSV 2.0, Single File, Parallel - Ordered by duration");

### Observations

Once again we see a similar profile to Parquet.
    
CSV 2.0 is nearly 35 seconds faster, on average, than Parquet.

### CSV 2.0, Multiple Files, Series

In [24]:
var csv20MultipleFilesSeriesResults = RunBenchmark("CSV 2.0, Muliple Files, Series", "SELECT * FROM NetworkHealth_MisonCSV_MultipleFiles", TestType.Series, force);

index,Name,RunNumber,DurationInSeconds
0,"CSV 2.0, Muliple Files, Series",1,21.995
1,"CSV 2.0, Muliple Files, Series",2,20.302
2,"CSV 2.0, Muliple Files, Series",3,18.981
3,"CSV 2.0, Muliple Files, Series",4,18.973
4,"CSV 2.0, Muliple Files, Series",5,19.092
5,"CSV 2.0, Muliple Files, Series",6,21.013
6,"CSV 2.0, Muliple Files, Series",7,20.259
7,"CSV 2.0, Muliple Files, Series",8,19.456
8,"CSV 2.0, Muliple Files, Series",9,22.324
9,"CSV 2.0, Muliple Files, Series",10,19.647


### Observations

Parquet outperformed CSV 2.0 over multiple files in series by ~ 2 seconds.

### CSV 2.0, Multiple Files, Parallel

In [25]:
var csv20MultipleFilesParallelResults = RunBenchmark("CSV 2.0, Muliple Files, Parallel", "SELECT * FROM NetworkHealth_MisonCSV_MultipleFiles", TestType.Parallel, force);

index,Name,RunNumber,DurationInSeconds
0,"CSV 2.0, Muliple Files, Parallel",1,42.913
1,"CSV 2.0, Muliple Files, Parallel",2,38.095
2,"CSV 2.0, Muliple Files, Parallel",3,32.866
3,"CSV 2.0, Muliple Files, Parallel",4,24.781
4,"CSV 2.0, Muliple Files, Parallel",5,56.899
5,"CSV 2.0, Muliple Files, Parallel",6,47.966
6,"CSV 2.0, Muliple Files, Parallel",7,64.92
7,"CSV 2.0, Muliple Files, Parallel",8,51.908
8,"CSV 2.0, Muliple Files, Parallel",9,53.773
9,"CSV 2.0, Muliple Files, Parallel",10,61.454


Order by duration

In [26]:
var r = csv20MultipleFilesParallelResults.OrderBy(o => o.DurationInSeconds).ToList();
Plot(r, "CSV 2.0, Multiple Files, Parallel - Ordered by duration");

### Observations

Parquet also outperforms CSV 2.0 over multiple files in parallel by ~ 19 seconds.

Similar profile to Parquet.

## Average duration

Work out and plot the average duration for all test runs.

In [27]:
var results = new IList<RadiusTestResult>[] 
{
    parquetSingleFileSeriesResults,
    parquetSingleFileParallelResults,
    parquetMultipleFilesSeriesResults,
    parquetMultipleFilesParallelResults,
    csvSingleFileSeriesResults,
    csvSingleFileParallelResults,
    csvMultipleFilesSeriesResults,
    csvMultipleFilesParallelResults,
    csv20SingleFileSeriesResults,
    csv20SingleFileParallelResults,
    csv20MultipleFilesSeriesResults,
    csv20MultipleFilesParallelResults,    
};

var summaryStats = results.SelectMany(o => o).GroupBy(r => r.Name, r => r.DurationInSeconds).Select(r => new { Name = r.Key, AverageDurationInSeconds = r.Average()}).OrderByDescending(r => r.AverageDurationInSeconds);
display(summaryStats);

var summaryChart = Chart.Plot(
        new [] {
            new Graph.Bar()
            {
                x = summaryStats.Select(o => o.Name),
                y = summaryStats.Select(o => o.AverageDurationInSeconds),
            }
        }
    );

var summaryLayout = new Layout.Layout()
{
    title = "Average duration in seconds",
    margin = new Graph.Margin {l = 20, r = 150, t=50, b=150}
};

summaryChart.WithLayout(summaryLayout);

display(summaryChart);



index,Name,AverageDurationInSeconds
0,"CSV, Single File, Parallel",225.89929999999998
1,"CSV, Single File, Series",101.81049999999999
2,"Parquet, Single File, Parallel",97.60700000000001
3,"CSV 2.0, Single File, Parallel",63.05230000000002
4,"CSV, Multiple Files, Parallel",47.6911
5,"CSV 2.0, Muliple Files, Parallel",47.557500000000005
6,"Parquet, Multiple Files, Parallel",29.0468
7,"Parquet, Single File, Series",27.919900000000002
8,"CSV, Multiple Files, Series",22.908499999999997
9,"CSV 2.0, Single File, Series",22.8675


### Series comparison Multiple Files

In [28]:

var parquetMultipleFilesSeriesSummary = parquetMultipleFilesSeriesResults.Skip(1).GroupBy(r => r.Name, r => r.DurationInSeconds).Select(r => new { Name = r.Key, AverageDurationInSeconds = r.Average()}).OrderByDescending(r => r.AverageDurationInSeconds);
var csvMultipleFilesSeriesSummary = csvMultipleFilesSeriesResults.GroupBy(r => r.Name, r => r.DurationInSeconds).Select(r => new { Name = r.Key, AverageDurationInSeconds = r.Average()}).OrderByDescending(r => r.AverageDurationInSeconds);
var csv20MulitpleFilesSeriesSummary = csv20MultipleFilesSeriesResults.GroupBy(r => r.Name, r => r.DurationInSeconds).Select(r => new { Name = r.Key, AverageDurationInSeconds = r.Average()}).OrderByDescending(r => r.AverageDurationInSeconds);

var seriesComparisonChart = Chart.Plot(
        new [] {
            new Graph.Bar()
            {
                x = parquetMultipleFilesSeriesSummary.Select(o => o.Name),
                y = parquetMultipleFilesSeriesSummary.Select(o => o.AverageDurationInSeconds),
            },            
            new Graph.Bar()
            {
                x = csvMultipleFilesSeriesSummary.Select(o => o.Name),
                y = csvMultipleFilesSeriesSummary.Select(o => o.AverageDurationInSeconds),
            },
            new Graph.Bar()
            {
                x = csv20MulitpleFilesSeriesSummary.Select(o => o.Name),
                y = csv20MulitpleFilesSeriesSummary.Select(o => o.AverageDurationInSeconds),
            }
        }
    );

var seriesComparisonLayout = new Layout.Layout()
{
    title = "Multiple files - average duration in seconds",
    margin = new Graph.Margin {l = 20, r = 150, t=50, b=150},
    showlegend = false
};

seriesComparisonChart.WithLayout(seriesComparisonLayout);

display(seriesComparisonChart);

In [29]:

var parquetSingleFileSeriesSummary = parquetSingleFileSeriesResults.Skip(1).GroupBy(r => r.Name, r => r.DurationInSeconds).Select(r => new { Name = r.Key, AverageDurationInSeconds = r.Average()}).OrderByDescending(r => r.AverageDurationInSeconds);
var csvSingleFileSeriesSummary = csvSingleFileSeriesResults.GroupBy(r => r.Name, r => r.DurationInSeconds).Select(r => new { Name = r.Key, AverageDurationInSeconds = r.Average()}).OrderByDescending(r => r.AverageDurationInSeconds);
var csv20SingleFileSeriesSummary = csv20SingleFileSeriesResults.GroupBy(r => r.Name, r => r.DurationInSeconds).Select(r => new { Name = r.Key, AverageDurationInSeconds = r.Average()}).OrderByDescending(r => r.AverageDurationInSeconds);

var seriesComparisonChart = Chart.Plot(
        new [] {
            new Graph.Bar()
            {
                x = parquetSingleFileSeriesSummary.Select(o => o.Name),
                y = parquetSingleFileSeriesSummary.Select(o => o.AverageDurationInSeconds),
            },            
            new Graph.Bar()
            {
                x = csvSingleFileSeriesSummary.Select(o => o.Name),
                y = csvSingleFileSeriesSummary.Select(o => o.AverageDurationInSeconds),
            },
            new Graph.Bar()
            {
                x = csv20SingleFileSeriesSummary.Select(o => o.Name),
                y = csv20SingleFileSeriesSummary.Select(o => o.AverageDurationInSeconds),
            }
        }
    );

var seriesComparisonLayout = new Layout.Layout()
{
    title = "Single file - average duration in seconds",
    margin = new Graph.Margin {l = 20, r = 150, t=50, b=150},
    showlegend = false
};

seriesComparisonChart.WithLayout(seriesComparisonLayout);

display(seriesComparisonChart);

# Summary

There are clearly several factors at play when determining the most suitable parser. 

Resource contention for parallel queries over the same file needs to be understood more.

Architectures that rely on responsive data processing may be suseptible to slower performance when multiple queries are run concurrently against the same file. The design of these solutions may need to build in some form of delay in order avoid the appent resource contention.

SQL Serverless pricing does not factor in duration only data volume scanned. Therefore the benefit of building in any delay to spread load and increase efficiency is likely to be offset by the delay itself. Also, since parser performance does not affect cost then the incentive to reduce resource ustilisation is low. This could result in higher costs to everyone using the service.

CSV 2.0 is far superior to CSV if you can live with the difference in featureset.

SQL Serverless offers a compelling alternative to Azure Data Lake Analytics. However, it does lack of extensibility features which will need to be considered.
